In [19]:
import numpy as np

In [20]:
class SVM:    
    def __init__(self, kernel='poly', degree=2, sigma=0.1, epoches=1000, learning_rate= 0.001):
            self.alpha = None
            self.b = 0
            self.degree = degree
            self.c = 1
            self.C = 1
            self.sigma = sigma
            self.epoches = epoches
            self.learning_rate = learning_rate
            
            if kernel == 'poly':
                    self.kernel = self.polynomial_kernal # for polynomial kernal
            elif kernel == 'rbf':
                    self.kernel =  self.gaussian_kernal # for guassian
                    
    def polynomial_kernal(self , x1 , x2):
        return (self.c + np.dot(x1,x2))**self.degree
    
    def gaussian_kernal(self , x1 ,x2):
        sigma_squared = np.power(self.sigma , 2)
        return np.exp(-(x1 - x2).T.dot(x1-x2)/(2*sigma_squared))
    
    def fit(self , X, y):
        self.X = X
        self.y = y
        self.alpha = np.random.random(X.shape[0])
        self.b = 0
        self.ones = np.ones(X.shape[0]) 
        
        y_mul_kernal = np.outer(y , y)*self.kernel(X,X)
        
        for _ in range(self.epoches):
            gradient = self.ones - y_mul_kernal.dot(self.alpha)
            
            self.alpha += self.learning_rate*(gradient)
            
            # self.alpha[self.alpha > self.C] = self.C # 0<α<C
            # self.alpha[self.alpha < 0] = 0 
            
            loss = np.sum(self.alpha) - 0.5 * np.sum(np.outer(self.alpha, self.alpha) * y_mul_kernal) # ∑αi – (1/2) ∑i ∑j αi αj yi yj K(xi, xj)
            alpha_index = np.where((self.alpha) > 0 & (self.alpha < self.C))[0]
            b_list = []        
            for index in alpha_index:
                b_list.append(y[index] - (self.alpha * y).dot(self.kernel(X, X[index])))

            self.b = np.mean(b_list)
            
        
        
        